In [2]:
!pip show webdriver_manager

Name: webdriver-manager
Version: 3.8.5
Summary: Library provides the way to automatically manage drivers for different browsers
Home-page: https://github.com/SergeyPirogov/webdriver_manager
Author: Sergey Pirogov
Author-email: automationremarks@gmail.com
License: 
Location: c:\users\admin\anaconda3\lib\site-packages
Requires: requests, python-dotenv, packaging, tqdm
Required-by: 


In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import lxml
import json
import time
import csv

In [3]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [5]:
service = Service(executable_path=ChromeDriverManager().install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:04<00:00, 1.51MB/s]


In [9]:
# Формируем суп-объект

def get_soup(url):    
    driver.get(url)    
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    return soup

In [10]:
# Извлекаем из soup-объекта название и ссылку с id фильма

def links_parse(row):
    page = []
    time.sleep(5)
    try:
        soup = get_soup(row)
        block_pos = soup.select("div.styles_upper__j8BIs a")
        for i in block_pos:    
            name_pos = i.select_one("div.base-movie-main-info_mainInfo__ZL_u3").get_text()
            link_pos = i.get('href')
            dic = {'name': name_pos, 'link': link_pos}  
            page.append(dic)
    except:
        page = []
    return page

In [11]:
# ЗАПУСК ПАРСИНГА
# - год меняется в теле ссылки после year--
# - количество страниц пагинации меняется в параметрах цикла range внутри скобок

link = 'https://www.kinopoisk.ru/lists/movies/year--2021/?page={}'
year = []
driver = webdriver.Chrome(service=service) # Открывает новую отдельную вкладку браузера хром от selenium
for i in range(1, 900):
    ln = link.format(i)
    inf = links_parse(ln)
    year.extend(inf)

In [12]:
# Записываем результаты в датафрейм

df_2021 = pd.DataFrame(year)

In [14]:
# Прибавляем хвост ссылки к домену

df_2021['links'] = df_2021['link'].apply(lambda x: 'https://www.kinopoisk.ru' + x)
df_2021 = df_2021[['name', 'links']]

In [15]:
df_2021

,name,links
0,Гнев человеческий,https://www.kinopoisk.ru/film/1318972/
1,Рик и Морти,https://www.kinopoisk.ru/series/685246/
2,Дюна,https://www.kinopoisk.ru/film/409424/
3,Лига справедливости Зака Снайдера,https://www.kinopoisk.ru/film/1387021/
4,Беспринципные,https://www.kinopoisk.ru/series/1355059/
...,...,...
44445,Never Ever,https://www.kinopoisk.ru/series/1366143/
44446,Гардемарины 1787. Мир,https://www.kinopoisk.ru/film/905024/
44447,Партия,https://www.kinopoisk.ru/series/957853/
44448,Крым,https://www.kinopoisk.ru/series/982486/


In [16]:
df_2021.to_csv('D:\\Data\\Kinopoisk\\df_2021.csv')

In [8]:
driver.quit()   # Закрывает отдельную вкладку браузера хром от selenium после выполнения цикла